# Giveaway prototype

In [51]:
import numpy as np
import pandas as pd
import math

In [34]:
rng_reproducible = np.random.default_rng(seed=42)
rng_fresh = np.random.default_rng()

Idea collection:
- load competitor list
- combine it with WCA DB info / competition info
- get newcomers
- get orga / delegates
- manually note down sponsorees who should not receive gift from sponsor (controversial)
- no repetition
- different modes:
    - treat everyone who is allowed to receive gift in same way
    - separate newcomers and returners (disjoint)
    - separate newcomers (group A) and everyone (group B)

In [35]:
# hard-coded stuff
WCA_database_path = '../wca-competition-orga/WCA_export.tsv/'

In [36]:
competitors_path = '../wca-competition-orga/DarmstadtDodecahedronDays2023-registration'
comp_id = 'DarmstadtDodecahedronDays2023'

In [37]:
# get competitor information for specific competition
competitors = pd.read_csv(f'{competitors_path}.csv')

In [38]:
names = competitors['Name']
countries = competitors['Country']
# replace NaN Ids with Str 'Newcomer' for proper printout
competitors['WCA ID'].replace(np.nan, 'Newcomer', inplace=True) # for all views of the column
wca_ids = competitors['WCA ID']
# get competition information (associated staff like organizers and delegates)
competitions = pd.read_csv(f'{WCA_database_path}WCA_export_Competitions.tsv',sep='\t')
this_comp = competitions[competitions['id'] == comp_id]

delegates = this_comp['wcaDelegate']
orga = this_comp['organiser']
comp_name = this_comp['name'].values[0]

# go from a series (pandas) to a list, then split until only names remain
delegates_ = [d for d in delegates]
delegates_ = delegates_[0].split('] [')
delegates_ = [d.split('}{')[0].split('{')[1] for d in delegates_]
orga_ = [o for o in orga]
orga_ = orga_[0].split('] [')
orga_ = [o.split('}{')[0].split('{')[1] for o in orga_]

potential_newcomers = [n for n in names[wca_ids == 'Newcomer']]

In [39]:
delegates_

['Annika Stein', 'Laura Holzhauer', 'Ricardo Olea Catalán']

In [40]:
orga_

['Alexander Botz',
 'Johanna Szczesny',
 'Kevin Gerhardt',
 'Malte Ihlefeld',
 'Wilhelm Kilders']

In [41]:
excludes = delegates_ + orga_

In [42]:
potential_newcomers

['Nicolas Walter',
 'Leni Schultheis',
 'Teo Kim',
 'Srinikshith Akula',
 'Sergei Shapovalov',
 'Yiming Jin',
 'Yihang Jin',
 'Manuel Kopp']

In [43]:
potential_names = [n for n in names if n not in excludes]

In [44]:
potential_names

['Achim Spies',
 'Patrick Hetco',
 'Finn Ickler',
 'Linh Son Luu',
 'Erik Akkersdijk',
 'Julian Hilpisch',
 'Eduardo Paez Rubio',
 'Andre Beer',
 'Henri Gerber',
 'Leon Schmidtchen',
 'Jonas Pilhöfer',
 'Samuel Francis Ballas',
 'Yannick Weller',
 'Nicolas Walter',
 'David Nesiba',
 'Mark Zimmermann',
 'David Leutert',
 'Duc Huynh',
 'Louis Stil',
 'Milan Wolfram',
 'Luc Prevost',
 'Jonathan Kobbe',
 'Konstantin Minkovic',
 'Nils Thorben Lang',
 'Alexis Fedeler',
 'Francesco Schmitt',
 'Yash Budhiraja',
 'Oscar Lin',
 'Sophia Schmoll',
 'Emma Pfeifer',
 'Wiebke Drude',
 'Titus Miholic',
 'Christian Beemelmann',
 'Berk Delen',
 'Jakob Schöley',
 'Nina Lutz',
 'Emilio Görg',
 'Ilhan M. Dumlupinar',
 'Eduardo Munive',
 'Thomas Miholic',
 'Richard Rettich',
 'Imanuel Leonid Müller',
 'Leni Schultheis',
 'Nico Maier',
 'Hendrik Ludwig',
 'Justas Indrisiunas',
 'Lucas Leon Erb',
 'Danny Kaiser',
 'Helmut Heilig',
 'Petr Samodelkin',
 'Yasin Kaan Ocakdan',
 'Tuna Kaya Erdal',
 'Oskar Herzog',

In [45]:
potential_returner = [r for r in ([n for n in names[wca_ids != 'Newcomer']]) if r not in excludes]

In [46]:
potential_returner

['Achim Spies',
 'Patrick Hetco',
 'Finn Ickler',
 'Linh Son Luu',
 'Erik Akkersdijk',
 'Julian Hilpisch',
 'Eduardo Paez Rubio',
 'Andre Beer',
 'Henri Gerber',
 'Leon Schmidtchen',
 'Jonas Pilhöfer',
 'Samuel Francis Ballas',
 'Yannick Weller',
 'David Nesiba',
 'Mark Zimmermann',
 'David Leutert',
 'Duc Huynh',
 'Louis Stil',
 'Milan Wolfram',
 'Luc Prevost',
 'Jonathan Kobbe',
 'Konstantin Minkovic',
 'Nils Thorben Lang',
 'Alexis Fedeler',
 'Francesco Schmitt',
 'Yash Budhiraja',
 'Oscar Lin',
 'Sophia Schmoll',
 'Emma Pfeifer',
 'Wiebke Drude',
 'Titus Miholic',
 'Christian Beemelmann',
 'Berk Delen',
 'Jakob Schöley',
 'Nina Lutz',
 'Emilio Görg',
 'Ilhan M. Dumlupinar',
 'Eduardo Munive',
 'Thomas Miholic',
 'Richard Rettich',
 'Imanuel Leonid Müller',
 'Nico Maier',
 'Hendrik Ludwig',
 'Justas Indrisiunas',
 'Lucas Leon Erb',
 'Danny Kaiser',
 'Helmut Heilig',
 'Petr Samodelkin',
 'Yasin Kaan Ocakdan',
 'Tuna Kaya Erdal',
 'Oskar Herzog',
 'Felix Stuhlmann',
 'Jordan Crippa',


In [53]:
mode = 'allInOne' # allInOne or newcomerThenReturner or newcomerThenAll
n_codes = 6

In [54]:
if mode == 'allInOne':
    draw_from = potential_names
    k = n_codes
elif mode == 'newcomerThenReturner':
    draw_from_A = potential_newcomers
    k_A = math.ceil(n_codes / 2.)
    draw_from_B = potential_returners
    k_B = math.floor(n_codes / 2.)
elif mode == 'newcomerThenAll':
    draw_from_A = potential_newcomers
    k_A = math.ceil(n_codes / 2.)
    #draw_from_B = potential_returners - drawn_in_stage_A

In [69]:
def draw_and_throw_out(list_to_draw_from):
    n = len(list_to_draw_from)
    random_i = int(rng_fresh.integers(low=0, high=n, size=1))
    the_chosen_one = list_to_draw_from[random_i]
    shortened_list = list_to_draw_from.copy()
    shortened_list.pop(random_i)
    return the_chosen_one, shortened_list

In [70]:
for c in range(k):
    chosen_name, new_list = draw_and_throw_out(draw_from)
    print(chosen_name, new_list, len(new_list))

Henri Gerber ['Achim Spies', 'Patrick Hetco', 'Finn Ickler', 'Linh Son Luu', 'Eduardo Paez Rubio', 'Leon Schmidtchen', 'Samuel Francis Ballas', 'David Nesiba', 'Mark Zimmermann', 'David Leutert', 'Duc Huynh', 'Milan Wolfram', 'Jonathan Kobbe', 'Nils Thorben Lang', 'Oscar Lin', 'Titus Miholic', 'Christian Beemelmann', 'Berk Delen', 'Jakob Schöley', 'Nina Lutz', 'Emilio Görg', 'Thomas Miholic', 'Nico Maier', 'Justas Indrisiunas', 'Danny Kaiser', 'Petr Samodelkin', 'Yasin Kaan Ocakdan', 'Jordan Crippa', 'Tobias Peter', 'Teo Kim', 'Eliah Mezler', 'Natan Suslowicz', 'Srinikshith Akula', 'József Katona', 'Linus Kunath', 'Yihang Jin', 'Manuel Kopp'] 37
Emilio Görg ['Achim Spies', 'Patrick Hetco', 'Finn Ickler', 'Linh Son Luu', 'Eduardo Paez Rubio', 'Henri Gerber', 'Leon Schmidtchen', 'Samuel Francis Ballas', 'David Nesiba', 'Mark Zimmermann', 'David Leutert', 'Duc Huynh', 'Milan Wolfram', 'Jonathan Kobbe', 'Nils Thorben Lang', 'Oscar Lin', 'Titus Miholic', 'Christian Beemelmann', 'Berk Delen'